Uploading pdf of book

In [ ]:
pip install pdfplumber

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [ ]:
from google.colab import files

uploaded = files.upload()


TypeError: 'NoneType' object is not subscriptable

**Part 1: Text to Speech**

In [ ]:
import os, requests, re, pathlib, time
import pdfplumber

API_KEY = "your_deepram_api_key"
MODEL = "aura-2-thalia-en"
PDF_PATH = "life3.0.pdf"

# Extracting text
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

TEXT = extract_text_from_pdf(PDF_PATH)

# Breaking into chunks
def chunk_sentences(t):
    chunks = re.split(r'(?<=[.!?])\s+', t)
    out, cur = [], ""
    for s in chunks:
        if len(cur) + len(s) < 2000:
            cur = (cur + " " + s).strip()
        else:
            if cur: out.append(cur)
            cur = s
    if cur: out.append(cur)
    return out

# Text to Speech
def tts_one(chunk, idx):
    url = "https://api.deepgram.com/v1/speak"
    headers = {"Authorization": f"Token {API_KEY}", "Content-Type": "application/json"}
    resp = requests.post(url, params={"model": MODEL}, headers=headers, json={"text": chunk})
    if resp.status_code != 200:
        print(f"Error {resp.status_code}: {resp.text}")
        return None
    out = f"part_{idx:03d}.mp3"
    with open(out, "wb") as f:
        f.write(resp.content)
    print(f"Saved {out}")
    return out


parts = []
for i, ch in enumerate(chunk_sentences(TEXT), 1):
    fn = tts_one(ch, i)
    if fn:
        parts.append(fn)
    time.sleep(0.2)

with open("parts.txt", "w") as f:
    for p in parts:
        f.write(f"file '{pathlib.Path(p).as_posix()}'\n")

print("\n✅ All chunks processed.")


In [ ]:
import os

with open("/content/parts.txt", "w") as f:
    for filename in sorted(os.listdir("/content")):
        if filename.startswith("part_") and filename.endswith(".mp3"):
            f.write(f"file '{filename}'\n")

print("✅ parts.txt created with the following content:\n")
print(open("/content/parts.txt").read())

# To merge
# !ffmpeg -f concat -safe 0 -i parts.txt -c copy life3_ai.mp3

**Part 2 : Speech to Speech**

In [ ]:
# Cloning OpenVoice repo
!rm -rf OpenVoice
!git clone https://github.com/myshell-ai/OpenVoice.git
%cd OpenVoice

# Installing required lib
!pip install unidecode eng_to_ipa inflect pypinyin jieba cn2an librosa pyworld gradio ffmpeg-python faster_whisper whisper_timestamped wavmark pydub

# Downloading Converter checkpoint
!mkdir -p checkpoints/converter
!wget -O checkpoints/converter/config.json https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/converter/config.json
!wget -O checkpoints/converter/checkpoint.pth https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/converter/checkpoint.pth

import sys, os
sys.path.append("/content/OpenVoice/OpenVoice")

# Connecting with Drive
from google.colab import drive
drive.mount('/content/drive')

# Setting paths
ref_file = "/content/drive/MyDrive/OpenVoice/myvoice.m4a"   # short reference voice
src_file = "/content/drive/MyDrive/OpenVoice/life3_ai.mp3" # text to speech audio
output_dir = "/content/drive/MyDrive/OpenVoice/cloned_chunks"

os.makedirs(output_dir, exist_ok=True)

# Speech to Speech
import torchaudio
import torch
from openvoice.api import ToneColorConverter
from openvoice import se_extractor
from pydub import AudioSegment
from pydub.utils import mediainfo

device = "cuda" if torch.cuda.is_available() else "cpu"

converter = ToneColorConverter("checkpoints/converter/config.json", device=device)
converter.load_ckpt("checkpoints/converter/checkpoint.pth")

tone_color, _ = se_extractor.get_se(ref_file, converter)

# Breaking into chunks as audio is long
info = mediainfo(src_file)
duration_ms = float(info['duration']) * 1000

chunk_length_ms = 5 * 60 * 1000  # 5 minutes

# detect how many chunks already processed
existing_chunks = sorted([f for f in os.listdir(output_dir) if f.endswith(".wav")])
start_idx = len(existing_chunks)
start = start_idx * chunk_length_ms
chunk_files = [os.path.join(output_dir, f) for f in existing_chunks]

print(f"🔹 Total duration: {duration_ms/1000/3600:.2f} hours")
print(f"⏩ Found {start_idx} chunks already in Drive, resuming from chunk {start_idx}...")

idx = start_idx
while start < duration_ms:
    chunk_path = f"/content/chunk_{idx:03d}.wav"
    cloned_path = os.path.join(output_dir, f"cloned_{idx:03d}.wav")

    # only load 5-min slice into memory
    chunk = AudioSegment.from_file(
        src_file,
        start_second=start/1000,
        duration=chunk_length_ms/1000
    )
    chunk.export(chunk_path, format="wav")

    # Convert with OpenVoice
    converter.convert(
        audio_src_path=chunk_path,
        src_se=None,
        tgt_se=tone_color,
        output_path=cloned_path
    )
    print(f"✅ Processed chunk {idx+1} -> {cloned_path}")
    chunk_files.append(cloned_path)

    start += chunk_length_ms
    idx += 1

print("🎉 All chunks processed and saved in Google Drive:", output_dir)


Cloning into 'OpenVoice'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 460 (delta 11), reused 7 (delta 7), pack-reused 445 (from 2)
Receiving objects: 100% (460/460), 3.85 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (218/218), done.
/content/OpenVoice
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 49.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


step59000_snr39.99_pesq4.35_BERP_none0.3(…):   0%|          | 0.00/10.0M [00:00<?, ?B/s]

...

/content/OpenVoice/openvoice/se_extractor.py:119: UserWarning: PySoundFile failed. Trying audioread instead.
  array, _ = librosa.load(audio_path, sr=None, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is no

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip
[(0.398, 17.344)]
after vad: dur = 16.946


/usr/local/lib/python3.12/dist-packages/torch/functional.py:730: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:875.)
  return _VF.stft(  # type: ignore[attr-defined]


🔹 Total duration: 12.58 hours
⏩ Found 12 chunks already in Drive, resuming from chunk 12...
✅ Processed chunk 13 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_012.wav
✅ Processed chunk 14 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_013.wav
✅ Processed chunk 15 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_014.wav
✅ Processed chunk 16 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_015.wav
✅ Processed chunk 17 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_016.wav
✅ Processed chunk 18 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_017.wav
✅ Processed chunk 19 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_018.wav
✅ Processed chunk 20 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_019.wav
✅ Processed chunk 21 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_020.wav
✅ Processed chunk 22 -> /content/drive/MyDrive/OpenVoice/cloned_chunks/cloned_021.wav
✅ Processed chunk 23 -> /content/drive/MyDrive/O

KeyboardInterrupt: 

In [ ]:
# Merging chunks
import os

output_dir = "/content/drive/MyDrive/OpenVoice/cloned_chunks"
merge_list = os.path.join(output_dir, "file_list.txt")

with open(merge_list, "w") as f:
    for idx in range(50):
        f.write(f"file '{output_dir}/cloned_{idx:03d}.wav'\n")

!ffmpeg -y -f concat -safe 0 -i $merge_list -c copy /content/drive/MyDrive/OpenVoice/merged.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab